## П1. Краевая задача в круге

1. Написать функцию, которая решает краевую задачу в круге радиуса $R$: 

$$
\begin{cases}
- \Delta u + {\alpha}u &= f(x,y),\quad  0 < x^2 + y^2 < R
\\
u\bigg|_{x^2 + y^2 = R,\; x < 0} &= h(x,y), 
\\
\frac{du}{dn}\bigg|_{x^2 + y^2 = R,\; x > 0} &= g(x,y), 
\end{cases}
$$

2. Для решения при помощи пакета `dolfinx` необходимо привести задачу к вариационной постановке следующего вида. Необходимо найти $u \in V$ такую, что 
$$
a(u, v) = L(v), \; \forall v \in V,
$$
где $V = H^1(\Omega)$ - функциональное пространство, а 
$$
\begin{align*}
a(u,v) &:= \int_\Omega \nabla u \nabla v \,dx + \alpha \int_\Omega u v \,dx
\\
L(v) &:= \int_\Omega f v \,dx + \int_{d \Omega} g  v \,ds.
\end{align*}
$$
Здесь 
$$
\Omega = \{(x,y): 0 < x^2 + y^2 < R \} \;\text{(круг радиуса $R$)},
$$
$$
d\Omega = \{(x,y):  x^2 + y^2 = R, \; x > 0 \} \; \text{("правая" граница круга)}
$$

In [1]:
import importlib.util

if importlib.util.find_spec("petsc4py") is not None:
    import dolfinx

from mpi4py import MPI
import numpy as np

import ufl
from dolfinx import fem, io, mesh, plot
from dolfinx.fem.petsc import LinearProblem
from ufl import ds, dx, grad, inner

import gmsh
gmsh.initialize()

import pyvista
from dolfinx.plot import vtk_mesh
pyvista.start_xvfb()

3. Выбор конкретных конечно-элементных пространств $V$  в FEniCS:

3.1 Задание расчетной области (сетки)

In [29]:
# Константы 

R = 1
alpha = 100

In [3]:
# Круг радиуса R с центром в начале координат
circle = gmsh.model.occ.addDisk(0, 0, 0, R, R)

gmsh.model.occ.synchronize()
gdim = 2
gmsh.model.addPhysicalGroup(gdim, [circle], 1)
gmsh.option.setNumber('Mesh.CharacteristicLengthMin', 0.05)
gmsh.option.setNumber('Mesh.CharacteristicLengthMax', 0.05)
gmsh.model.mesh.generate(gdim)

# Сетка
mesh, _, _ = io.gmshio.model_to_mesh(gmsh.model, MPI.COMM_WORLD, 0, gdim=gdim)
# Координаты сетки
x = ufl.SpatialCoordinate(mesh)

Info    : Meshing 1D...
Info    : Meshing curve 1 (Ellipse)
Info    : Done meshing 1D (Wall 0.000191671s, CPU 0.000129s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0410288s, CPU 0.040927s)
Info    : 1550 nodes 3099 elements


3.2 Задание типа функциональных пространств (степень и тип)

In [4]:
V = fem.functionspace(mesh, ('Lagrange', 1))

left_boundary = fem.locate_dofs_geometrical(
    V, lambda x: x[0] < 0 & np.isclose(np.sqrt(x[0]**2 + x[1]**2), R))

4. Решить задачу, визуализировать решение, посчитать погрешность в узлах сетки.

4.1 Выбор тестовых функций

Выберем следующие тестовые функции. В первую очередь выбирается $h(x,y)$. Например, как 

$$
h(x,y) = \sin(x) * \cos(y).
$$

Если $h(x,y)$ является точным решением $u(x,y)$ поставленной задачи, то $g(x,y)$ и $f(x,y)$ вычисляются следующим способом:

$$
\begin{align*}
f & =-\Delta u + {\alpha}u = -\Delta h + \alpha h,\\ 
g & ={du \over dn} = {dh \over dn}.
\end{align*}
$$

Итого имеем:
$$
\begin{align*}
h(x,y) &= \sin(x) * \cos(y), \\ 
f(x,y) & = 2 * \cos(y) * \sin(x) + \alpha * \sin(x) * \cos(y), \\
g(x,y) & =  \cos(x) * \cos(y) - \sin(x) * \sin(y).
\end{align*}
$$


In [13]:
# x[0] ~ x, x[1] ~ y

h = ufl.sin(x[0]) * ufl.cos(x[1])
f = 2 * ufl.cos(x[1]) * ufl.sin(x[0]) + alpha * ufl.sin(x[0]) * ufl.cos(x[1])
g = ufl.cos(x[0]) * ufl.cos(x[1]) - ufl.sin(x[0]) * ufl.sin(x[1])

4.2 Решение задачи

In [14]:
# Задаем граничное условие Дирихле u = h(x,y) на "левой" границе круга
u_lbc = fem.Function(V)
u_lbc.interpolate(fem.Expression(h, V.element.interpolation_points()))
lbc = fem.dirichletbc(u_lbc, left_boundary)

In [15]:
# Задаем дискретную вариационную задачу
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx + alpha * ufl.dot(u, v) * ufl.dx
L = f * v * ufl.dx + g * v * ufl.ds
problem = LinearProblem(a, L, bcs=[lbc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

In [16]:
# Получаем численное решение
uh = problem.solve()

4.3 Визуализация численного решения

In [17]:
pyvista.set_jupyter_backend('client')

pyvista_cells, cell_types, geometry = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, geometry)
grid.point_data['u'] = uh.x.array
grid.set_active_scalars('u')

plotter = pyvista.Plotter()
plotter.add_text('uh', position='upper_edge', font_size=14, color='black')
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x71e196328c70_2" src="http://95.131.149.198:1489/…

4.4 Получаем точное решение $u(x,y)$ как интерполяцию $h(x,y)$ на сетке

In [18]:
u_sol = fem.Function(V)
u_sol.interpolate(fem.Expression(h, V.element.interpolation_points()))

In [19]:
pyvista.set_jupyter_backend('client')

pyvista_cells, cell_types, geometry = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, geometry)
grid.point_data['u'] = u_sol.x.array
grid.set_active_scalars('u')

plotter = pyvista.Plotter()
plotter.add_text('u', position='upper_edge', font_size=14, color='black')
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x71e196328760_3" src="http://95.131.149.198:1489/…

4.5 Вычисляем погрешности

In [20]:
# Ошибка по норме L2
error_L2 = fem.assemble_scalar(fem.form((uh - u_sol)**2 * ufl.dx))
error_L2 = np.sqrt(MPI.COMM_WORLD.allreduce(error_L2, op=MPI.SUM))

# Ошибка по максимум-норме
u_vertex_values = uh.x.array
u_sol_vertex_values = u_sol.x.array
error_max = np.max(np.abs(u_vertex_values - u_sol_vertex_values))
error_max = MPI.COMM_WORLD.allreduce(error_max, op=MPI.MAX)

print ('Отклонение численного решения от точного по максимум-норме: ', error_max)
print ('Отклонение численного решения от точного по L2-норме: ', error_L2)

Отклонение численного решения от точного по максимум-норме:  0.09962952782504031
Отклонение численного решения от точного по L2-норме:  0.020969557529437733


5. Другие тестовые функции $h(x,y)$

$$
\begin{align*}
h(x,y) &= e^{x^2 + y^2}, \\ 
f(x,y) & = -4 e^{x^2 + y^2} * (1 + x^2 + y^2) + \alpha * e^{x^2 + y^2}, \\
g(x,y) & =  2* x* e^{x^2 + y^2} + 2 * y * e^{x^2 + y^2}
\end{align*}
$$


In [30]:
# x[0] ~ x, x[1] ~ y

h = ufl.exp(x[0] ** 2 + x[1] ** 2)
f = - 4 * ufl.exp(x[0] ** 2 + x[1] ** 2) * (1 + x[0]**2 + x[1]**2) + alpha * ufl.exp(x[0] ** 2 + x[1] ** 2)
g = 2 * x[0] * ufl.exp(x[0] ** 2 + x[1] ** 2) + 2 * x[1] * ufl.exp(x[0] ** 2 + x[1] ** 2)

# Задаем граничное условие Дирихле u = h(x,y) на "левой" границе круга
u_lbc = fem.Function(V)
u_lbc.interpolate(fem.Expression(h, V.element.interpolation_points()))
lbc = fem.dirichletbc(u_lbc, left_boundary)

# Задаем дискретную вариационную задачу
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx + alpha * ufl.dot(u, v) * ufl.dx
L = f * v * ufl.dx + g * v * ufl.ds
problem = LinearProblem(a, L, bcs=[lbc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

# Получаем численное решение
uh = problem.solve()

pyvista.set_jupyter_backend('client')

pyvista_cells, cell_types, geometry = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, geometry)
grid.point_data['u'] = uh.x.array
grid.set_active_scalars('u')

plotter = pyvista.Plotter()
plotter.add_text('uh', position='upper_edge', font_size=14, color='black')
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x71e196328280_8" src="http://95.131.149.198:1489/…

In [31]:
u_sol = fem.Function(V)
u_sol.interpolate(fem.Expression(h, V.element.interpolation_points()))

pyvista.set_jupyter_backend('client')

pyvista_cells, cell_types, geometry = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, geometry)
grid.point_data['u'] = u_sol.x.array
grid.set_active_scalars('u')

plotter = pyvista.Plotter()
plotter.add_text('u', position='upper_edge', font_size=14, color='black')
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x71e195c0e7a0_9" src="http://95.131.149.198:1489/…

In [32]:
# Ошибка по норме L2
error_L2 = fem.assemble_scalar(fem.form((uh - u_sol)**2 * ufl.dx))
error_L2 = np.sqrt(MPI.COMM_WORLD.allreduce(error_L2, op=MPI.SUM))

# Ошибка по максимум-норме
u_vertex_values = uh.x.array
u_sol_vertex_values = u_sol.x.array
error_max = np.max(np.abs(u_vertex_values - u_sol_vertex_values))
error_max = MPI.COMM_WORLD.allreduce(error_max, op=MPI.MAX)

print ('Отклонение численного решения от точного по максимум-норме: ', error_max)
print ('Отклонение численного решения от точного по L2-норме: ', error_L2)

Отклонение численного решения от точного по максимум-норме:  0.9415374703775696
Отклонение численного решения от точного по L2-норме:  0.17294199122130177


$$
\begin{align*}
h(x,y) &= x^2 + y, \\ 
f(x,y) & = -2 + \alpha * x^2 + y, \\
g(x,y) & =  2* x* y + x ^ 2
\end{align*}
$$


In [42]:
# x[0] ~ x, x[1] ~ y

h = x[0]**2 + x[1]
f = -2 + alpha * x[0]**2 + x[1]
g = 2 * x[0] * x[1] + x[0] ** 2

# Задаем граничное условие Дирихле u = h(x,y) на "левой" границе круга
u_lbc = fem.Function(V)
u_lbc.interpolate(fem.Expression(h, V.element.interpolation_points()))
lbc = fem.dirichletbc(u_lbc, left_boundary)

# Задаем дискретную вариационную задачу
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx + alpha * ufl.dot(u, v) * ufl.dx
L = f * v * ufl.dx + g * v * ufl.ds
problem = LinearProblem(a, L, bcs=[lbc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

# Получаем численное решение
uh = problem.solve()

pyvista_cells, cell_types, geometry = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, geometry)
grid.point_data['u'] = uh.x.array
grid.set_active_scalars('u')

plotter = pyvista.Plotter()
plotter.add_text('uh', position='upper_edge', font_size=14, color='black')
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x71e193ed6d70_17" src="http://95.131.149.198:1489…

In [43]:
u_sol = fem.Function(V)
u_sol.interpolate(fem.Expression(h, V.element.interpolation_points()))

pyvista.set_jupyter_backend('client')

pyvista_cells, cell_types, geometry = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, geometry)
grid.point_data['u'] = u_sol.x.array
grid.set_active_scalars('u')

plotter = pyvista.Plotter()
plotter.add_text('u', position='upper_edge', font_size=14, color='black')
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x71e193fbfdf0_18" src="http://95.131.149.198:1489…

In [44]:
# Ошибка по норме L2
error_L2 = fem.assemble_scalar(fem.form((uh - u_sol)**2 * ufl.dx))
error_L2 = np.sqrt(MPI.COMM_WORLD.allreduce(error_L2, op=MPI.SUM))

# Ошибка по максимум-норме
u_vertex_values = uh.x.array
u_sol_vertex_values = u_sol.x.array
error_max = np.max(np.abs(u_vertex_values - u_sol_vertex_values))
error_max = MPI.COMM_WORLD.allreduce(error_max, op=MPI.MAX)

print ('Отклонение численного решения от точного по максимум-норме: ', error_max)
print ('Отклонение численного решения от точного по L2-норме: ', error_L2)

Отклонение численного решения от точного по максимум-норме:  0.8497513529514247
Отклонение численного решения от точного по L2-норме:  0.5280650175349745


Проверка на адекватность

$$
\begin{align*}
h(x,y) &= x, \\ 
f(x,y) & =  \alpha * x, \\
g(x,y) & =  1
\end{align*}
$$


In [45]:
# x[0] ~ x, x[1] ~ y

h = x[0]
f = alpha * x[0]
g = 1

# Задаем граничное условие Дирихле u = h(x,y) на "левой" границе круга
u_lbc = fem.Function(V)
u_lbc.interpolate(fem.Expression(h, V.element.interpolation_points()))
lbc = fem.dirichletbc(u_lbc, left_boundary)

# Задаем дискретную вариационную задачу
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx + alpha * ufl.dot(u, v) * ufl.dx
L = f * v * ufl.dx + g * v * ufl.ds
problem = LinearProblem(a, L, bcs=[lbc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

# Получаем численное решение
uh = problem.solve()

pyvista_cells, cell_types, geometry = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, geometry)
grid.point_data['u'] = uh.x.array
grid.set_active_scalars('u')

plotter = pyvista.Plotter()
plotter.add_text('uh', position='upper_edge', font_size=14, color='black')
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x71e193e5efe0_19" src="http://95.131.149.198:1489…

In [46]:
u_sol = fem.Function(V)
u_sol.interpolate(fem.Expression(h, V.element.interpolation_points()))

pyvista.set_jupyter_backend('client')

pyvista_cells, cell_types, geometry = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, geometry)
grid.point_data['u'] = u_sol.x.array
grid.set_active_scalars('u')

plotter = pyvista.Plotter()
plotter.add_text('u', position='upper_edge', font_size=14, color='black')
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x71e193e30e80_20" src="http://95.131.149.198:1489…

In [47]:
# Ошибка по норме L2
error_L2 = fem.assemble_scalar(fem.form((uh - u_sol)**2 * ufl.dx))
error_L2 = np.sqrt(MPI.COMM_WORLD.allreduce(error_L2, op=MPI.SUM))

# Ошибка по максимум-норме
u_vertex_values = uh.x.array
u_sol_vertex_values = u_sol.x.array
error_max = np.max(np.abs(u_vertex_values - u_sol_vertex_values))
error_max = MPI.COMM_WORLD.allreduce(error_max, op=MPI.MAX)

print ('Отклонение численного решения от точного по максимум-норме: ', error_max)
print ('Отклонение численного решения от точного по L2-норме: ', error_L2)

Отклонение численного решения от точного по максимум-норме:  0.07813769208861635
Отклонение численного решения от точного по L2-норме:  0.016696281171242
